<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [58]:
import pandas as pd
import psycopg2

In [59]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [60]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query = "SELECT COUNT(*) FROM vacancies"
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\3654173318.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,49197


In [16]:
query = "SELECT COUNT(*) FROM employers"
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_14480\447867848.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,23501


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [10]:
# текст запроса
query = "SELECT COUNT(*) as employers_count FROM employers"
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\3389115295.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employers_count
0,23501


In [ ]:
23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query = "SELECT COUNT(*) as regions_count FROM areas"
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\179194388.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,regions_count
0,1362


In [ ]:
1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [12]:
query = "SELECT COUNT(*) as industries_count FROM industries"
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\3408430778.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,industries_count
0,294


In [ ]:
294

***

In [ ]:
Общее количество вакансий: 49 197 — данных достаточно для репрезентативный анализ рынка 

Количество работодателей: 23 501 — почти в два раза меньше количества вакансий, что говорит о том, что в среднем один работодатель размещает около 2 вакансий. 

ГЕография поиска
Количество регионов: 1362 — включаем города, села России и стран СНГ.

Количество сфер деятельности: 294 — это показывает высокую диверсификацию отраслей, представленных в базе. Рынок вакансий не ограничивается только IT или традиционными секторами, а включает широкий спектр профессиональных областей.


Соотношение вакансий к работодателям: ~2.1 , как описал выше, значит:
среднюю или высокую конкуренцию среди работодателей за кандидатов
есть малые предприятия или стартапы

Соотношение регионов к вакансиям: ~36 вакансий на регион в среднем, но распределение, как показывает следующий анализ, крайне неравномерное.



# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
query = """
SELECT 
    a.name as region,
    COUNT(v.id) as cnt
FROM vacancies v
JOIN areas a ON v.area_id = a.id
GROUP BY a.id, a.name
ORDER BY cnt DESC
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\914782387.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,region,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


In [ ]:
Москва, Санкт-Петербург, Минск, Новосибирск, Алматы

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query = """
SELECT 
    COUNT(*) as vacancies_with_salary
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\4111096268.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_with_salary
0,24073


In [ ]:
24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [15]:
query = """
SELECT 
    ROUND(AVG(salary_from)) as avg_salary_from,
    ROUND(AVG(salary_to)) as avg_salary_to
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2262933605.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [ ]:
71065 ; 110537

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [16]:
query = """
SELECT 
    schedule,
    employment,
    COUNT(*) as vacancy_count
FROM vacancies
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\1666821935.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,schedule,employment,vacancy_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


In [ ]:
Удаленная работа,Полная занятость

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [17]:
query = """
SELECT 
    experience,
    COUNT(*) as vacancy_count
FROM vacancies
GROUP BY experience
ORDER BY vacancy_count ASC
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\401293027.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,experience,vacancy_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [ ]:
Более 6 лет,        1337
Нет опыта,          7197
От 3 до 6 лет,      14511
От 1 года до 3 лет, 26152


***

In [ ]:
Высокая концентрация в столицах: Москва (5333 вакансии), Санкт-Петербург (2851) и Минск (2112) лидируют с большим отрывом. Рынок сосредоточен в городах милионниках. Догоняют Новосиб и Алматы
Далее идет существенный отскос на 1-10 вакансий, что говорит о низком развитии и стабильности рынка и в целом большой фрагментации рынка труда 

Большое кол-во работодателей не хотят раскрывать зарплаты для соискателей. Только 48,9% вакансий (24 073 из 49 197)

Формы занятости и графики работы
Чудес не происодит, удаленка пока не побеждает ): Сочетание «Полный день + Полная занятость» (35 367 вакансий). Удаленка составляет всего  (7 802) — второй по популярности вариант, что отражает тренд на дистанционный формат.
Но появляются гибкие форматы : «Гибкий график + Полная занятость» (1 593) и «Удалённая работа + Частичная занятость» (1 312) 
Стажировки: В основном предлагаются с полным днём (569), что типично для программ обучения молодых специалистов.

Наибольший спрос на специалистов среднего уровня: Вакансии с опытом «От 1 года до 3 лет» (26 152) и «От 3 до 6 лет» (14 511) составляют большинство.

Низкий спрос на высокий опыт: «Более 6 лет» — всего 1 337 вакансий. Это может означать:

Либо спрос маленький, либо дорого, либо закрытый подбор


Подытоги
Централизация рынка: Основные возможности сосредоточены в столицах и крупных городах.
Проблема прозрачности зарплат: Почти половина вакансий не содержит зарплатной информации.
Традиционный формат доминирует: Полная занятость с полным днём остаётся нормой, несмотря на рост удалённой работы.
Спрос на опытных специалистов: Рынок ориентирован на кандидатов с опытом от 1 до 6 лет.

Хочешь работу - будеь готов переезжать в крупный город
Наибольший скачек может быть с опытом от года до трех 
ну и конечно офис )


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [18]:
query = """
WITH employer_vacancies AS (
    SELECT 
        e.name as employer_name,
        COUNT(v.id) as vacancy_count,
        ROW_NUMBER() OVER (ORDER BY COUNT(v.id) DESC, e.name) as rank
    FROM employers e
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
)
SELECT 
    employer_name,
    vacancy_count,
    rank
FROM employer_vacancies
WHERE rank = 1 OR rank = 5
ORDER BY rank
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2847292950.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employer_name,vacancy_count,rank
0,Яндекс,1933,1
1,Газпром нефть,331,5


In [ ]:
Яндекс и Газпром

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [21]:
query = """
SELECT 
    a.name as region,
    COUNT(DISTINCT e.id) as employers_count
FROM areas a
LEFT JOIN employers e ON a.id = e.area
WHERE a.id NOT IN (
    SELECT DISTINCT area_id 
    FROM vacancies 
    WHERE area_id IS NOT NULL
)
GROUP BY a.id, a.name
ORDER BY COUNT(DISTINCT e.id) DESC
LIMIT 1
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\1244950629.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,region,employers_count
0,Россия,410


In [ ]:
Россия 410

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [22]:
query = """
WITH employer_regions AS (
    SELECT 
        e.id,
        e.name as employer_name,
        COUNT(DISTINCT v.area_id) as region_count
    FROM employers e
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
)
SELECT 
    employer_name,
    region_count
FROM employer_regions
WHERE region_count = (SELECT MAX(region_count) FROM employer_regions)
ORDER BY employer_name
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\242128707.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employer_name,region_count
0,Яндекс,181


In [ ]:
Яндекс - 181

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [24]:
query = """
SELECT COUNT(*) as employers_without_industry
FROM employers e
WHERE e.id NOT IN (
    SELECT employer_id 
    FROM employers_industries 
    WHERE industry_id IS NOT NULL
)
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\700051700.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employers_without_industry
0,8419


In [ ]:
8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [25]:
query = """
WITH companies_with_four_industries AS (
    SELECT 
        e.id,
        e.name as company_name,
        COUNT(ei.industry_id) as industry_count
    FROM employers e
    JOIN employers_industries ei ON e.id = ei.employer_id
    GROUP BY e.id, e.name
    HAVING COUNT(ei.industry_id) = 4
)
SELECT 
    company_name
FROM companies_with_four_industries
ORDER BY company_name
LIMIT 1 OFFSET 2
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2457067454.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,company_name
0,2ГИС


In [ ]:
2ГИС

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [26]:
query = """
SELECT COUNT(DISTINCT ei.employer_id) as employers_count
FROM employers_industries ei
JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
"""
df = pd.read_sql_query(query, connection)
df

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2949777999.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employers_count
0,3553


In [ ]:
3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [51]:
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
cities_list = []

headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers, timeout=10)
response.encoding = 'utf-8'
html = response.text

soup = BeautifulSoup(html, 'html.parser')

for table in soup.find_all('table'):
    for th in table.find_all('th'):
        if 'город' in th.get_text(strip=True).lower():
            for row in table.find_all('tr')[1:]:
                cells = row.find_all('td')
                if len(cells) > 1:
                    city_name = cells[1].get_text(strip=True).split('[')[0].strip()
                    if city_name:
                        cities_list.append(city_name)
            break
    if cities_list:
        break

for city in cities_list:
    print(city)

Москва
Санкт-Петербург
Новосибирск
Екатеринбург
Казань
Красноярск
Нижний Новгород
Челябинск
Уфа
Краснодар
Самара
Ростов-на-Дону
Омск
Воронеж
Пермь
Волгоград


In [54]:

# Формируем строку для SQL-запроса
cities_sql_list = ", ".join([f"'{city}'" for city in cities_list])

# SQL-запрос 
sql_query = f"""
WITH yandex_vacancies AS (
    SELECT 
        a.name AS city,
        COUNT(v.id) AS cnt
    FROM vacancies v
    JOIN employers e ON v.employer_id = e.id
    JOIN areas a ON v.area_id = a.id
    WHERE e.name = 'Яндекс' 
      AND a.name IN ({cities_sql_list})
    GROUP BY a.id, a.name
)
SELECT city, cnt FROM yandex_vacancies
UNION ALL
SELECT 'Total' AS city, SUM(cnt) AS cnt FROM yandex_vacancies
ORDER BY cnt ASC;
"""

df = pd.read_sql_query(sql_query, connection)

print(df.to_string(index=False))

           city   cnt
           Омск  21.0
      Челябинск  22.0
     Красноярск  23.0
      Волгоград  24.0
 Ростов-на-Дону  25.0
         Казань  25.0
          Пермь  25.0
            Уфа  26.0
         Самара  26.0
      Краснодар  30.0
        Воронеж  32.0
    Новосибирск  35.0
Нижний Новгород  36.0
   Екатеринбург  39.0
Санкт-Петербург  42.0
         Москва  54.0
          Total 485.0


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2796752497.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, connection)


In [ ]:
      city   cnt
           Омск  21.0
      Челябинск  22.0
     Красноярск  23.0
      Волгоград  24.0
 Ростов-на-Дону  25.0
         Казань  25.0
          Пермь  25.0
            Уфа  26.0
         Самара  26.0
      Краснодар  30.0
        Воронеж  32.0
    Новосибирск  35.0
Нижний Новгород  36.0
   Екатеринбург  39.0
Санкт-Петербург  42.0
         Москва  54.0
          Total 485.0

***

In [ ]:
Ничего удивительного, но абсолютный лидер Яндекс — 1933 вакансии, что в 5,8 раз больше, чем у пятого места
Газпром нефть — 5-е место (331 вакансия)
Вывод  - IT-сектор самый активный
причем Яндекс имеет самый широкий охват, прям реклама яндекса.
3553 работодателя указали "Разработку ПО" как сферу — самая популярная отрасль
многие компании зарегистрированы, но не публикуют вакансии открыто - вероятно закрытый поиск или все стабильно, или наоборот упадок


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [61]:

sql_query = """
SELECT COUNT(*) as data_vacancies_count
FROM vacancies 
WHERE name ILIKE '%data%' OR name ILIKE '%данн%';
"""

try:
    result_df = pd.read_sql_query(sql_query, connection)
    print(f"Вакансий, связанных с данными: {result_df['data_vacancies_count'].iloc[0]}")
except Exception as e:
    print(f"Ошибка: {e}")

Вакансий, связанных с данными: 1771


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\4242676220.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(sql_query, connection)


In [ ]:
1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [65]:
sql_query = """
SELECT COUNT(*) as junior_ds_vacancies
FROM vacancies 
WHERE (
    -- Условия для Data Science вакансий
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%'
    OR (
        LOWER(name) LIKE '%ml%' 
        AND LOWER(name) NOT LIKE '%html%'  -- Исключаем HTML
    )
)
AND (
    -- Условия для Junior уровня
    LOWER(name) LIKE '%junior%'
    OR LOWER(experience) = 'нет опыта'
    OR LOWER(employment) = 'стажировка'
);
"""
try:
    result_df = pd.read_sql_query(sql_query, connection)
    count = result_df['junior_ds_vacancies'].iloc[0]
    print(f"Вакансий для начинающего дата-сайентиста: {count}")

except Exception as e:
    print(f"Ошибка при выполнении запроса: {e}")

Вакансий для начинающего дата-сайентиста: 51


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2501419593.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(sql_query, connection)


In [ ]:
51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [66]:
sql_query = """
SELECT COUNT(*) as ds_sql_vacancies
FROM vacancies 
WHERE (
    -- Условия для Data Science вакансий
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%'
    OR (
        LOWER(name) LIKE '%ml%' 
        AND LOWER(name) NOT LIKE '%html%'
    )
)
AND (
    -- Условия для навыков SQL или Postgres
    LOWER(key_skills) LIKE '%sql%'
    OR LOWER(key_skills) LIKE '%postgres%'
);
"""

try:
    result_df = pd.read_sql_query(sql_query, connection)
    count = result_df['ds_sql_vacancies'].iloc[0]
    print(f"Количество DS-вакансий с навыком SQL/Postgres: {count}")
except Exception as e:
    print(f"Ошибка при выполнении запроса: {e}")

Количество DS-вакансий с навыком SQL/Postgres: 229


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\4035414939.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(sql_query, connection)


In [ ]:
229

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [67]:
sql_query = """
SELECT COUNT(*) as ds_python_vacancies
FROM vacancies 
WHERE (
    -- Условия для Data Science вакансий
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%'
    OR (
        LOWER(name) LIKE '%ml%' 
        AND LOWER(name) NOT LIKE '%html%'
    )
)
AND (
    -- Условие для навыка Python
    LOWER(key_skills) LIKE '%python%'
);
"""

try:
    result_df = pd.read_sql_query(sql_query, connection)
    count = result_df['ds_python_vacancies'].iloc[0]
    print(f"Количество DS-вакансий с навыком Python: {count}")
except Exception as e:
    print(f"Ошибка при выполнении запроса: {e}")

Количество DS-вакансий с навыком Python: 357


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2529138611.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(sql_query, connection)


In [ ]:
357

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [85]:
query_skills = """

SELECT ROUND(
    AVG(
        array_length(
            array_remove(string_to_array(key_skills, CHR(9)), ''),
            1
        )
    )::NUMERIC,
    2
) AS avg_key_skills

FROM vacancies
WHERE (
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%'
    OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND key_skills IS NOT NULL
AND key_skills != '';
"""


df_skills = pd.read_sql_query(query_skills, connection)

print(f"Среднее количество навыков: {df_skills['avg_key_skills'].values[0]}")

C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\4127846544.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_skills = pd.read_sql_query(query_skills, connection)


Среднее количество навыков: 6.55


In [ ]:
6.55  не совпадаем.. уже не знаю почему, либо кто-то данные поменял, либо ошибка у вас, но я одурел  искать истину )))

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [88]:
query_salary = """
SELECT
    experience,
    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2.0)) AS avg_salary
FROM vacancies
WHERE (
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%'
    OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience;

"""
df_salary = pd.read_sql_query(query_salary, connection)
print(df_salary)

 

           experience  avg_salary
0         Более 6 лет    157933.0
1       От 3 до 6 лет    256454.0
2           Нет опыта     74643.0
3  От 1 года до 3 лет    144233.0


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\2227390113.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_salary = pd.read_sql_query(query_salary, connection)


In [ ]:
256454    - короче у вас ошибка ))

***

In [80]:
stats_query = """
SELECT 
    COUNT(*) as total_ds_vacancies,
    MIN(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1) as min_skills,
    MAX(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1) as max_skills,
    AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1) as avg_skills_raw,
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1), 2) as avg_skills_rounded
FROM vacancies 
WHERE (
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%'
    OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND key_skills IS NOT NULL 
AND key_skills != '';
"""

print("\n" + "="*60)
print("Статистика по текущей формуле подсчёта:")
stats_df = pd.read_sql_query(stats_query, connection)
print(stats_df.to_string(index=False))



Статистика по текущей формуле подсчёта:
 total_ds_vacancies  min_skills  max_skills  avg_skills_raw  avg_skills_rounded
                481           1          20        6.546778                6.55


C:\Users\rooot\AppData\Local\Temp\ipykernel_26088\477212976.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stats_df = pd.read_sql_query(stats_query, connection)


# Общий вывод по проекту

In [ ]:
по Data Science - 3,6% от всех вакансий, достаточно большой сегмент от ИТ доли
Навыки у ДС Рython — критически важный навык 20% DS-сегмента
SQL/Postgres — 13%
так же можно отметить низкий порог вхождения для новичков: всего 51 вакансия для Junior-специалистов
Зарплатные ожидания 3-6 лет опыта — пик зарплат 256 454 (видимо у вас ошибка? :))))
Странно, но тут ошибки данных, либо закрытая информация, т.к. специалисты 6+ лет получают меньше 157 933, чем 3-6 лет


и общие вывод:
Рынок вакансий HeadHunter  — большинство предложений сосредоточено в Москве, Санкт-Петербурге и столицах СНГ, при этом лидируют IT-компании во главе с Яндексом. Для Data Science-сегмента критически важны навыки Python и SQL, но возможности для начинающих ограничены, а зарплатный пик достигается после 3–6 лет опыта. Значительная часть данных неполна: почти у половины вакансий не указана зарплата, а у трети работодателей не заполнена сфера деятельности. Традиционный формат полной занятости превалирует, однако удалённая работа стала набирает обороты. В целом рынок сформирован вокруг крупных IT-работодателей
